# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

The chain usually combines an LLM, large language model, 
together with a prompt, and with this building block you 
can also put a bunch of these building blocks 
together to carry out a sequence of operations on your text or 
on your other data. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
#!pip install pandas

In [ ]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [ ]:
df.head()

| Product                | Review                                                                                           |
|------------------------|--------------------------------------------------------------------------------------------------|
| Queen Size Sheet Set   | I ordered a king size set. My only criticism was that the sheets were a bit too thin for my liking. |
| Waterproof Phone Pouch | I loved the waterproof pouch, although the opening was a little tricky to seal at times.        |
| Luxury Air Mattress    | This mattress had a small hole in the top of it, causing a bit of discomfort during use.        |
| Pillows Insert         | This is the best throw pillow filler on Amazon, provides great support and fluffiness.         |
| Milk Frother Handheld  | I loved this product. But they only seem to last for a few months before breaking.              |


## LLMChain

So the first chain we're going to cover is the LLM chain. And 
this is a simple but really powerful chain 
that underpins a lot of the chains that we'll go 
over in the future. 
And so, we're going to import three different things. We're going 
to import the OpenAI model, so the LLM. We're going to import the chat prompt 
template. And so this is the prompt. And then we're 
going to import the LLM chain. 

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

We're going 
to initialize the language model that we want 
to use. So we're going to initialize the 
chat OpenAI with a high temperature so that we can get 
some fun descriptions. 

In [ ]:
llm = ChatOpenAI(temperature=0.9)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

LLM chain = It's just the combination of the LLM 
and the prompt.

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = "Queen Size Sheet Set"
chain.run(product)
# it will format the prompt under the hood, and then it will pass the whole prompt into the LLM.

*OUTPUT*

'Royal Rest'

![Chain](immagini/08_chain.png)

So the LLM chain is the most basic type of chain. 
And that's going to be used a lot in the future. And 
so we can see how this will be used in 
the next type of chain, which will be sequential chains. And 
so sequential chains run a sequence of chains 
one after another. 

## SimpleSequentialChain

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
overall_simple_chain.run(product)

*OUTPUT*
```
> Entering new SimpleSequentialChain chain...
"Regal Rest Linens"
Regal Rest Linens offers luxurious and comfortable bed linens, ensuring a peaceful and restful night's sleep for customers.

> Finished chain.
"Regal Rest Linens offers luxurious and comfortable bed linens, ensuring a peaceful and restful night's sleep for customers."
```

The simple sequential chain works well when there's 
only a single input and a single output. 
But what about when there are multiple inputs or multiple outputs? 
And so we can do this by using just the regular sequential chain. 

## SequentialChain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [ ]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


One important thing to note about all these subchains 
is that the input keys and output keys 
need to be pretty precise.

![Chain](immagini/09_chain.png)

 The simple sequential chain takes in multiple chains, 
where each one has a single input and a single output. 
To see a visual representation of this, we can look at the slide, 
where it has one chain 
feeding into the other chain, one after another. 
Here we can see a visual description of the sequential chain. 
Comparing it to the above chain, you can notice that any 
step in the chain can take in multiple input variables. 
This is useful when you have more complicated downstream 
chains that need to be a composition of multiple 
previous chains. 

We have all these chains, we can easily combine 
them in the sequential chain. You'll notice here that we'll pass 
in the four chains we created into the 
chains variable. We'll create the inputs variable with the one 
human input, which is the review. 
And then we want to return all the intermediate outputs. 
So the English review, the summary, and then the follow-up message. 

In [ ]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

We can run this over some of the data. 

In [ ]:
review = df.Review[5]
overall_chain(review)

```
> Entering new SequentialChain chain...

> Finished chain.
{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't last, it's weird. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit?!",
 'summary': 'The reviewer is disappointed with the mediocre taste, lack of foam longevity, and suspects the possibility of receiving either an old batch or counterfeit product compared to the better-tasting ones purchased from physical stores.',
 'followup_message': "Réponse: Cher(e) client(e),\n\nNous vous remercions d'avoir partagé votre expérience avec notre produit. Nous sommes désolés d'apprendre que vous avez été déçu(e) par le goût moyen et la faible longévité de la mousse. Nous tenons à vous assurer que nous prenons vos commentaires très au sérieux.\n\nIl est important pour nous de fournir à nos clients des produits de haute qualité. Nous nous excusons si vous avez reçu un produit qui ne correspondait pas à vos attentes. Si vous pensez avoir reçu un vieux lot ou une contrefaçon, nous vous encourageons vivement à nous contacter directement afin de résoudre ce problème.\n\nNous avons à cœur de satisfaire nos clients et nous aimerions avoir l'opportunité de rectifier cette situation. Votre satisfaction est notre priorité.\n\nEncore une fois, nous vous remercions d'avoir pris le temps de partager votre avis. Nous espérons avoir l'occasion de regagner votre confiance et de vous offrir une expérience positive à l'avenir.\n\nCordialement,\nL'équipe du service client"}
 ```

A pretty common but basic operation is to 
route an input to a chain depending on 
what exactly that input is. 
A good way to imagine this is if you have multiple sub chains, 
each of which specialized for a particular type of input, 
you could have a router chain which first 
decides which subchain to pass it to and then passes it to 
that chain. 

![Chain](immagini/11_chain.png)

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

Here we need a multi-prompt chain. 
This is a specific type of chain that is used when routing 
between multiple different prompt templates. 


LLM router chain: 
This uses a language model itself to route 
between the different subchains. This is where the 
description and the name provided above will be used. 
We'll also import a router output parser. 
This parses the LLM output into a dictionary 
that can be used downstream to determine which 
chain to use and what the input to that chain should be. 

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

# LLMRouterChain = This uses a language model itself to route between the different subchains.
# This is where the description and the name provided above will be used. 
# RouterOutputParser =  This parses the LLM output into a dictionary that can be used downstream to determine which 
# chain to use and what the input to that chain should be.

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

These are the chains 
that will be called by the router chain. 
As you can see, each destination chain itself 
is a language model chain, an LLM chain. 
In addition to the destination chains, we also need a default chain. 
This is the chain that's called when the router can't decide 
which of the subchains to use.

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

Note that here we have the router output parser. 
This is important as it will help this chain 
decide which subchains to route between. 
And finally, putting it all together, we can create 
the overall chain. 

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")

![Chain](immagini/12_chain.png)

In [ ]:
chain.run("what is 2 + 2")

![Chain](immagini/13_chain.png)

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

![Chain](immagini/14_chain.png)